In [9]:
import numpy as np
from scipy.optimize import curve_fit, minimize
from scipy.stats import chi2
from scipy.interpolate import interp1d
import os
import sys
import argparse
import matplotlib.pyplot as plt

# # Argument parser setup
# parser = argparse.ArgumentParser()
# parser.add_argument('--args.cov_type', help='mocks or cosmolike', default='mocks', type=str)
# parser.add_argument('--args.cosmology_template', help='mice, planck or lognormal_Y6BAO', default='mice', type=str)
# parser.add_argument('--args.cosmology_covariance', help='mice, planck or lognormal_Y6BAO', default='mice', type=str)
# parser.add_argument('--args.diag_only', help='y or n', default='n', type=str)
# parser.add_argument('--args.remove_crosscov', help='y or n', default='n', type=str)
# parser.add_argument('--args.delta_theta', help='Values: 0.05, 0.1, 0.15 or 0.2', default=0.2, type=float)
# parser.add_argument('--args.theta_min', help='min theta to do the BAO fit', default=0.5, type=float)
# parser.add_argument('--args.theta_max', help='max theta to do the BAO fit', default=5, type=float)
# parser.add_argument('--args.n_broadband', help='1, 2, 3, 4 or 5', default=3, type=int)
# parser.add_argument('--args.bins_removed', help='bins removed when doing the BAO fit', default='None', type=str)
# parser.add_argument('--args.nz_flag', help='original or modified (shifted+stretched)', default='fid', type=str)
# parser.add_argument('--args.include_wiggles', help='y or n', default='y', type=str)
# parser.add_argument('--args.weight_type', help='Only allowed if fit_data=1. 0 for no weights, 1 for systematic weights applied', default=1, type=int)

# args = parser.parse_args()

# Arguments
class Args:
    def __init__(self):
        self.cov_type = "cosmolike_data"
        self.cosmology_template = "planck"
        self.cosmology_covariance = "planck"
        self.diag_only = "n"
        self.remove_crosscov = "n"
        self.delta_theta = 0.2
        self.theta_min = 0
        self.theta_max = 5
        self.n_broadband = 3
        self.bins_removed = 'None'
        self.nz_flag = "fid"
        self.include_wiggles = "y"
        self.weight_type = 1
# class Args:
#     def __init__(self):
#         self.cov_type = "cosmolike_data"
#         self.cosmology_template = "planck"
#         self.cosmology_covariance = "planck"
#         self.diag_only = "n"
#         self.remove_crosscov = "n"
#         self.delta_theta = 0.2
#         self.theta_min = 0
#         self.theta_max = 5
#         self.n_broadband = 6
#         self.bins_removed = 'None'
#         self.nz_flag = "fid_5"
#         self.include_wiggles = "n"
#         self.weight_type = 1
args = Args()
args.include_wiggles = '' if args.include_wiggles == 'y' else '_noBAO'

# Limits for the fits
alpha_min_allowed = 0.8
alpha_max_allowed = 1.2

# Handling args.bins_removed argument
bin_mappings = {
    'None': [],
    '0': [0], '1': [1], '2': [2], '3': [3], '4': [4], '5': [5],
    '25': [2, 5], '12345': [1, 2, 3, 4, 5], '02345': [0, 2, 3, 4, 5],
    '01345': [0, 1, 3, 4, 5], '01245': [0, 1, 2, 4, 5], '01235': [0, 1, 2, 3, 5], '0125': [0, 1, 2, 5],
    '01234': [0, 1, 2, 3, 4], '012': [0, 1, 2], '345': [3, 4, 5], '45': [4, 5]
}
args.bins_removed = bin_mappings.get(args.bins_removed, args.bins_removed)

# Galaxy bias dictionary
galaxy_bias = {0: 1, 1: 1, 2: 1, 3: 1, 4: 1, 5: 1}

# Redshift distributions
from utils import redshift_distributions
nz_instance = redshift_distributions(args.nz_flag)

# Setting up save directory
savedir = (
    f"fit_results{args.include_wiggles}/data_{args.weight_type}/nz{args.nz_flag}_cov{args.cov_type}_"
    f"{args.cosmology_template}temp_{args.cosmology_covariance}cov_deltatheta{args.delta_theta}_"
    f"thetamin{args.theta_min}_{args.n_broadband}broadband_binsremoved{args.bins_removed}"
)

# # Check if the results already exist
# if os.path.exists(f"{savedir}/alpha_results_data.txt"):
#     print("This one already done, moving to the next one!")
#     sys.exit()

# Create directory if it doesn't exist
os.makedirs(savedir, exist_ok=True)

# Load the covariance matrix
if args.cov_type == 'cosmolike_data':
    if args.cosmology_covariance == 'mice':
        if args.delta_theta != 0.2:
            print(f"No mice cosmolike covariance matrix for delta_theta={args.delta_theta}")
            sys.exit()
        cov = np.loadtxt(
            f"cov_cosmolike/cov_Y6bao_data_DeltaTheta{str(args.delta_theta).replace('.', 'p')}_mask_g_mice.txt"
        )
    elif args.cosmology_covariance == 'planck':
        cov = np.loadtxt(
            f"cov_cosmolike/cov_Y6bao_data_DeltaTheta{str(args.delta_theta).replace('.', 'p')}_mask_g_planck.txt"
        )
    theta_cov = np.loadtxt(f"cov_cosmolike/delta_theta_{args.delta_theta}_binning.txt")[:, 2] * np.pi / 180

# Adjust covariance matrix for removed bins
cov_adjusted = np.zeros_like(cov)
for bin_z1 in range(nz_instance.nbins):
    for bin_z2 in range(nz_instance.nbins):
        slice_1 = slice(bin_z1 * len(theta_cov), (bin_z1 + 1) * len(theta_cov))
        slice_2 = slice(bin_z2 * len(theta_cov), (bin_z2 + 1) * len(theta_cov))
        if bin_z1 == bin_z2 or (bin_z1 not in args.bins_removed and bin_z2 not in args.bins_removed):
            cov_adjusted[slice_1, slice_2] = cov[slice_1, slice_2]
cov = cov_adjusted

# Remove cross-covariances
if args.remove_crosscov == 'y':
    cov_adjusted = np.zeros_like(cov)
    for bin_z in range(nz_instance.nbins):
        slice_ = slice(bin_z * len(theta_cov), (bin_z + 1) * len(theta_cov))
        cov_adjusted[slice_, slice_] = cov[slice_, slice_]
    cov = cov_adjusted

# Retain only the diagonal
if args.diag_only == 'y':
    cov = np.diag(np.diag(cov))

cov_orig = np.copy(cov)

# Extend theta covariance across all bins
theta_cov = np.concatenate([theta_cov] * nz_instance.nbins)








# Load the data and theoretical wtheta
indices = {}
indices_2 = {}

theta_wtheta_data = {}
wtheta_data = {}

theta_wtheta_th = {}
wtheta_th = {}

wtheta_bb_th = {}
wtheta_bf_th = {}
wtheta_ff_th = {}

err_wtheta_data = {}

for bin_z in range(nz_instance.nbins):
    filename_wtheta = f'wtheta_data_Yband/wtheta_data_bin{bin_z}_DeltaTheta{args.delta_theta}_weights{args.weight_type}_fstar.txt'
    theta, wtheta = np.loadtxt(filename_wtheta).T
    
    ind_theta = np.where((theta > args.theta_min * np.pi / 180) & (theta < args.theta_max * np.pi / 180))[0]

    theta_wtheta_data[bin_z] = theta[ind_theta]
    wtheta_data[bin_z] = wtheta[ind_theta]

    templates_base = f'wtheta_template{args.include_wiggles}/nz_{args.nz_flag}/wtheta_{args.cosmology_template}'
    theta_wtheta_th[bin_z] = np.loadtxt(f'{templates_base}/wtheta_bb_bin{bin_z}.txt')[:, 0]
    wtheta_bb_th[bin_z] = np.loadtxt(f'{templates_base}/wtheta_bb_bin{bin_z}.txt')[:, 1]
    wtheta_bf_th[bin_z] = np.loadtxt(f'{templates_base}/wtheta_bf_bin{bin_z}.txt')[:, 1]
    wtheta_ff_th[bin_z] = np.loadtxt(f'{templates_base}/wtheta_ff_bin{bin_z}.txt')[:, 1]

    wtheta_th[bin_z] = (
        galaxy_bias[bin_z]**2 * wtheta_bb_th[bin_z] +
        galaxy_bias[bin_z] * wtheta_bf_th[bin_z] +
        wtheta_ff_th[bin_z]
    )

    if bin_z == 0:
        indices[bin_z] = [0, len(ind_theta)]
    else:
        indices[bin_z] = [indices[bin_z - 1][1], indices[bin_z - 1][1] + len(ind_theta)]

    indices_2[bin_z] = ind_theta + bin_z * len(theta)

    err_wtheta_data[bin_z] = np.sqrt(np.diag(cov_orig))[indices_2[bin_z]]

# Interpolate the theoretical wtheta to use it as the template
wtheta_th_interp = {}
wtheta_bb_th_interp = {}
wtheta_bf_th_interp = {}
wtheta_ff_th_interp = {}

for bin_z in range(nz_instance.nbins):
    wtheta_th_interp[bin_z] = interp1d(theta_wtheta_th[bin_z], wtheta_th[bin_z], kind='cubic')
    wtheta_bb_th_interp[bin_z] = interp1d(theta_wtheta_th[bin_z], wtheta_bb_th[bin_z], kind='cubic')
    wtheta_bf_th_interp[bin_z] = interp1d(theta_wtheta_th[bin_z], wtheta_bf_th[bin_z], kind='cubic')
    wtheta_ff_th_interp[bin_z] = interp1d(theta_wtheta_th[bin_z], wtheta_ff_th[bin_z], kind='cubic')

# Prepare the covariance matrix
indices_fit = np.concatenate([indices_2[i] for i in range(nz_instance.nbins)])
cov = cov_orig[indices_fit[:, None], indices_fit]
inv_cov = np.linalg.inv(cov)

# Concatenate the datavector
theta_wtheta_data_concatenated = np.concatenate([theta_wtheta_data[i] for i in range(nz_instance.nbins)])

wtheta_data_concatenated = []
for bin_z in range(nz_instance.nbins):
    if bin_z in args.bins_removed:
        wtheta_data_concatenated.append(np.zeros(len(wtheta_data[bin_z])))
    else:
        wtheta_data_concatenated.append(wtheta_data[bin_z])
wtheta_data_concatenated = np.concatenate(wtheta_data_concatenated)

err_wtheta_data_concatenated = np.concatenate([err_wtheta_data[i] for i in range(nz_instance.nbins)])

# Verify consistency between theta from covariance and data
print('This should be 0:', abs(theta_wtheta_data_concatenated - theta_cov[indices_fit]).max())
if abs(theta_wtheta_data_concatenated - theta_cov[indices_fit]).max() > 1e-5:
    print('The covariance matrix and the wtheta of the mocks do not have the same theta')
    sys.exit()










# Parameter names. Max 6 broadband terms, max 6 redshift bins. Can be adjusted
name = np.array([
    'alpha',
    'A_0', 'B_0', 'C_0', 'D_0', 'E_0', 'F_0', 'G_0',
    'A_1', 'B_1', 'C_1', 'D_1', 'E_1', 'F_1', 'G_1',
    'A_2', 'B_2', 'C_2', 'D_2', 'E_2', 'F_2', 'G_2',
    'A_3', 'B_3', 'C_3', 'D_3', 'E_3', 'F_3', 'G_3',
    'A_4', 'B_4', 'C_4', 'D_4', 'E_4', 'F_4', 'G_4',
    'A_5', 'B_5', 'C_5', 'D_5', 'E_5', 'F_5', 'G_5',
])

n_broadband_max = int((len(name) - 1)/6 - 1) # this should be 6
n_params_max = len(name)

# Initial parameter guesses and bounds
p0_list = [1]
for _ in range(nz_instance.nbins):
    row = [1] + [0] * (n_broadband_max)
    p0_list.extend(row)

p0 = np.array(p0_list)

bounds = (
    (alpha_min_allowed, *[0, -1, -1, -1, -1, -1, -1] * nz_instance.nbins),
    (alpha_max_allowed, *[15, 1, 1, 1, 1, 1, 1] * nz_instance.nbins)
)

IND = np.arange(1, 1 + (args.n_broadband + 1))
IND2 = np.concatenate([[0], IND])

for k in range(1, nz_instance.nbins):
    IND2 = np.concatenate([IND2, IND + k * (1 + n_broadband_max)])

name = name[IND2]
p0 = p0[IND2]
bounds = (
    list(np.array(bounds[0])[IND2]),
    list(np.array(bounds[1])[IND2])
)

# Calculate effective parameter counts
n_params = len(p0)
n_params_eff = len(p0) - (1 + args.n_broadband) * len(args.bins_removed)

# Define the theoretical template

def wtheta_template_raw(x, alpha, *params):
    x_sections = [theta_wtheta_data_concatenated[indices[i][0]:indices[i][1]] for i in range(nz_instance.nbins)]

    for i, x_i in enumerate(x_sections):
        if (alpha * x_i).min() < theta_wtheta_th[i].min() or \
           (alpha * x_i).max() > theta_wtheta_th[i].max():
            print('Bad news: the template is being extrapolated')
            print(alpha)

    v = np.concatenate([
        params[(1 + n_broadband_max) * i] * wtheta_th_interp[i](alpha * x_sections[i]) + # constant that multiplies the template (A)
        params[(1 + n_broadband_max) * i + 1] + # constant (B)
        params[(1 + n_broadband_max) * i + 2] / x_sections[i] + # 1/theta term (C)
        params[(1 + n_broadband_max) * i + 3] / x_sections[i]**2 + # 1/theta**2 term (D)
        params[(1 + n_broadband_max) * i + 4] * x_sections[i] + # theta term (E)
        params[(1 + n_broadband_max) * i + 5] * x_sections[i]**2 + # theta**2 term (F)
        params[(1 + n_broadband_max) * i + 6] * x_sections[i]**3 # theta**3 term (G)
        for i in range(nz_instance.nbins)
    ])

    return v

def get_wtheta_template(n_broadband, n_params_max, IND2):
    param_names = []
    for i in range(args.n_broadband + 1):
        for letter in 'ABCDEFG':
            param_names.append(f"{letter}_{i}")

    def wtheta_template(x, *args):
        pars = np.zeros(n_params_max)
        pars[IND2] = args
        return wtheta_template_raw(x, *pars)
    
    return wtheta_template

wtheta_template = get_wtheta_template(args.n_broadband, n_params_max, IND2)

# First fit attempt. We use curve_fit

popt, pcov = curve_fit(wtheta_template, theta_wtheta_data_concatenated, wtheta_data_concatenated, p0=p0, bounds=bounds, sigma=cov, absolute_sigma=False)
err_params = np.sqrt(np.diag(pcov))















# Let's compute the chi-squared statistic
wtheta_fit_concatenated = wtheta_template(theta_wtheta_data_concatenated, *popt)
diff_mean = wtheta_data_concatenated - wtheta_fit_concatenated
chi_square = diff_mean @ inv_cov @ diff_mean

def least_squares(params):
    y_th = wtheta_template(theta_wtheta_data_concatenated, *params)
    diff = wtheta_data_concatenated - y_th
    return diff @ inv_cov @ diff

# Initialize the model matrix MM
MM = np.zeros([len(theta_wtheta_data_concatenated), nz_instance.nbins * args.n_broadband])

# Positions of the amplitude parameters in the model function
pos_amplitude = np.array([1 + i * (args.n_broadband + 1) for i in np.arange(0, nz_instance.nbins)])

# Positions of the broadband parameters in the model function
pos_broadband = np.delete(np.arange(0, n_params), np.concatenate(([0], pos_amplitude)))

# Construct the model matrix MM
for j in np.arange(0, nz_instance.nbins * args.n_broadband):
    fit_params = np.zeros(n_params)
    fit_params[0] = 1  # The value of alpha doesn't matter here since the amplitude is 0
    fit_params[pos_broadband[j]] = 1
    MM[:, j] = wtheta_template(theta_wtheta_data_concatenated, *fit_params)

# Compute the pseudo-inverse of MM
MM_2 = np.linalg.inv((MM.T) @ inv_cov @ MM) @ (MM.T) @ inv_cov


def broadband_params(amplitude_params, alpha):
    fit_params = np.zeros(n_params)
    fit_params[0] = alpha
    fit_params[pos_amplitude] = amplitude_params
    return MM_2 @ (wtheta_data_concatenated - wtheta_template(theta_wtheta_data_concatenated, *fit_params))


def least_squares_2(amplitude_params, alpha):
    fit_params = np.zeros(n_params)
    fit_params[0] = alpha
    fit_params[pos_amplitude] = amplitude_params
    fit_params[pos_broadband] = broadband_params(amplitude_params, alpha)

    vector_ones = np.ones(nz_instance.nbins)
    vector_ones[args.bins_removed] = 0

    return least_squares(fit_params) + np.sum(((amplitude_params - vector_ones) / 0.4) ** 2)


# Tolerance for the minimization procedure
tol_minimize = 1e-7

# Search for the alpha that minimizes the chi-squared over a grid from alpha_min_allowed to alpha_max_allowed
n = 2 * 10**2
alpha_vector = np.linspace(alpha_min_allowed, alpha_max_allowed, n)
chi2_vector = np.zeros(n)

for i in np.arange(0, n):
    amplitude_params = minimize(least_squares_2, x0=np.ones(nz_instance.nbins), method='SLSQP', bounds=([(0, None)] * nz_instance.nbins), 
                                tol=tol_minimize, args=(alpha_vector[i]))
    chi2_vector[i] = least_squares_2(amplitude_params.x, alpha_vector[i])

# Find the best alpha value
best = np.argmin(chi2_vector)
alpha_best = alpha_vector[best]
chi2_best = chi2_vector[best]

# Set chi-square to the best chi-squared value
chi_square = chi2_best

# Save the likelihood data for '_noBAO' condition
if args.include_wiggles == '_noBAO':
    np.savetxt(savedir + '/likelihood_data.txt', np.column_stack([alpha_vector, chi2_vector]))

elif args.include_wiggles == '':
    if chi2_vector[0] > chi2_best + 1 and chi2_vector[-1] > chi2_best + 1:
        
        for i in np.arange(0, best)[::-1]:
            if chi2_vector[i] < chi2_best + 1 and chi2_vector[i - 1] > chi2_best + 1:
                alpha_down = alpha_vector[i]
                break

        for i in np.arange(best, n):
            if chi2_vector[i] < chi2_best + 1 and chi2_vector[i + 1] > chi2_best + 1:
                alpha_up = alpha_vector[i]
                break

        alpha_best_vector = np.linspace(alpha_best - 0.01, alpha_best + 0.01, n)
        chi2_best_vector = np.zeros(n)
        alpha_down_vector = np.linspace(alpha_down - 0.01, alpha_down + 0.01, n)
        chi2_down_vector = np.zeros(n)
        alpha_up_vector = np.linspace(alpha_up - 0.01, alpha_up + 0.01, n)
        chi2_up_vector = np.zeros(n)

        for i in np.arange(0, n):
            amplitude_params_best = minimize(least_squares_2, x0=np.ones(nz_instance.nbins), method='SLSQP', bounds=([(0, None)] * nz_instance.nbins),
                                             tol=tol_minimize, args=(alpha_best_vector[i]))
            chi2_best_vector[i] = least_squares_2(amplitude_params_best.x, alpha_best_vector[i])

            amplitude_params_down = minimize(least_squares_2, x0=np.ones(nz_instance.nbins), method='SLSQP', bounds=([(0, None)] * nz_instance.nbins),
                                             tol=tol_minimize, args=(alpha_down_vector[i]))
            chi2_down_vector[i] = least_squares_2(amplitude_params_down.x, alpha_down_vector[i])

            amplitude_params_up = minimize(least_squares_2, x0=np.ones(nz_instance.nbins), method='SLSQP', bounds=([(0, None)] * nz_instance.nbins),
                                           tol=tol_minimize, args=(alpha_up_vector[i]))
            chi2_up_vector[i] = least_squares_2(amplitude_params_up.x, alpha_up_vector[i])

        best_new = np.argmin(chi2_best_vector)
        alpha_best_new = alpha_best_vector[best_new]
        chi2_best_new = chi2_best_vector[best_new]

        alpha_down_new = alpha_down_vector[np.argmin(abs(chi2_down_vector - (chi2_best + 1)))]
        alpha_up_new = alpha_up_vector[np.argmin(abs(chi2_up_vector - (chi2_best + 1)))]
        
        if alpha_best_vector[0] < alpha_best_new < alpha_best_vector[-1] and \
           alpha_down_vector[0] < alpha_down_new < alpha_down_vector[-1] and \
           alpha_up_vector[0] < alpha_up_new < alpha_up_vector[-1]:
            alpha_best, alpha_down, alpha_up = alpha_best_new, alpha_down_new, alpha_up_new
            chi_square = chi2_best_new
        else:
            print('There is a problem with the fit!')
            good_fit = 1

        amplitude_params = minimize(least_squares_2, x0=np.ones(nz_instance.nbins), method='SLSQP', bounds=([(0, None)] * nz_instance.nbins),
                                    tol=tol_minimize, args=(alpha_best))

        popt = np.zeros(n_params)
        popt[0] = alpha_best
        popt[pos_amplitude] = amplitude_params.x
        popt[pos_broadband] = broadband_params(amplitude_params.x, alpha_best)

        err_alpha_down = alpha_best - alpha_down
        err_alpha_up = alpha_up - alpha_best
        err_alpha = (alpha_up - alpha_down) / 2
        err_params[0] = err_alpha

        fig = plt.figure()
        plt.plot(alpha_vector, chi2_vector)
        plt.plot(alpha_best, chi2_best, 'd')
        plt.plot(alpha_vector, np.ones(n) * (chi2_best + 1), '--r')
        plt.plot((alpha_best - err_alpha_down) * np.ones(n), np.linspace(chi2_vector.min(), chi2_vector.max(), n), '--k')
        plt.plot((alpha_best + err_alpha_up) * np.ones(n), np.linspace(chi2_vector.min(), chi2_vector.max(), n), '--k')
        plt.xlabel(r'$\alpha$', fontsize=14)
        plt.ylabel(r'$\chi^2$', fontsize=14)
        plt.savefig(savedir + '/fig.png', bbox_inches='tight')
        plt.close(fig)

        np.savetxt(savedir + '/likelihood_data.txt', np.column_stack([alpha_vector, chi2_vector]))

    else:
        print('The fit does not have the 1-sigma region between ' + str(alpha_min_allowed) + ' and ' + str(alpha_max_allowed))
        good_fit = 1

        fig = plt.figure()
        plt.plot(alpha_vector, chi2_vector)
        plt.plot(alpha_best, chi2_best, 'd')
        plt.plot(alpha_vector, np.ones(n) * (chi2_best + 1), '--r')
        plt.xlabel(r'$\alpha$', fontsize=14)
        plt.ylabel(r'$\chi^2$', fontsize=14)
        plt.savefig(savedir + '/fig_bad.png', bbox_inches='tight')
        plt.close(fig)

wtheta_fit_concatenated = wtheta_template(theta_wtheta_data_concatenated, *popt)
alpha = popt[0]
p0 = popt
popt_mean = popt
err_params_mean = err_params

dof = len(indices_fit)
for i in np.arange(0, len(args.bins_removed)):
    dof = dof - len(indices_2[args.bins_removed[i]])
dof = dof - n_params_eff

p_value = chi2.sf(chi_square, dof)

alpha_mean_mocks = alpha
err_alpha_mean_mocks = err_params[0]

results = [round(alpha_mean_mocks, 4),
           round(err_alpha_mean_mocks, 4),
           round(chi_square, 4),
           round(dof, 4)]

print(results)

np.savetxt(savedir + '/alpha_results_data.txt', results, fmt='%1.4f', newline=' ')
np.savetxt(savedir + '/wtheta_data_fit.txt', np.column_stack([err_wtheta_data_concatenated, wtheta_data_concatenated, wtheta_fit_concatenated, err_wtheta_data_concatenated]))
np.save(savedir + '/indices_fit.npy', indices)


Loading fid n(z), which has 6 redshift bins
This should be 0: 2.7755575615628914e-17
[0.9513, 0.0282, 94.7796, 119]


In [ ]:
[0.9837, 0.0536, 27.135, 33]


In [ ]:
[0.9449, 0.0331, 74.6871, 84]